In [1]:
import numpy as np
from functools import partial
import old_vis_utils as viz_utils
from iris_plant_visualizer_old import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display

In [2]:
#pydrake imports
import pydrake
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
from pydrake.solvers import mathematicalprogram as mp
from pydrake.all import RigidTransform, RollPitchYaw, RevoluteJoint

import pydrake.multibody.rational as rational_forward_kinematics
from pydrake.all import RationalForwardKinematics
from pydrake.geometry.optimization import (#IrisOptionsRationalSpace, 
                                           IrisInRationalConfigurationSpace, 
                                           HPolyhedron, 
                                           Hyperellipsoid,
                                           Iris, IrisOptions)
from dijkstraspp import DijkstraSPPsolver           

<ipython-input-2-f0a98216a54b>:8: DrakeDeprecationWarning: Please import from the pydrake.solvers module directly, instead of the deprecated pydrake.solvers.mathematicalprogram submodule. The deprecated code will be removed from Drake on or after 2023-05-01.
  from pydrake.solvers import mathematicalprogram as mp


In [3]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
oneDOF_iiwa_asset = "../../drake/C_Iris_Examples/assets/oneDOF_iiwa7_with_box_collision.sdf"#FindResourceOrThrow("drake/C_Iris_Examples/assets/oneDOF_iiwa7_with_box_collision.sdf")
twoDOF_iiwa_asset = "../../drake/C_Iris_Examples/assets/twoDOF_iiwa7_with_box_collision.sdf"#FindResourceOrThrow("drake/C_Iris_Examples/assets/twoDOF_iiwa7_with_box_collision.sdf")

box_asset = "../../drake/C_Iris_Examples/assets/box_small.urdf" #FindResourceOrThrow("drake/C_Iris_Examples/assets/box_small.urdf")

models = []
models.append(parser.AddModelFromFile(box_asset))
models.append(parser.AddModelFromFile(twoDOF_iiwa_asset))
models.append(parser.AddModelFromFile(oneDOF_iiwa_asset))



locs = [[0.,0.,0.],
        [0.,.55,0.],
        [0.,-.55,0.]]
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("base", models[0]),
                 RigidTransform(locs[0]))
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("iiwa_twoDOF_link_0", models[1]), 
                 RigidTransform(RollPitchYaw([0,0, -np.pi/2]).ToRotationMatrix(), locs[1]))
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("iiwa_oneDOF_link_0", models[2]), 
                 RigidTransform(RollPitchYaw([0,0, -np.pi/2]).ToRotationMatrix(), locs[2]))


plant.Finalize()

idx = 0
q0 = [0.0, 0.0, 0.0]
q_low  = [-1.7, -2., -1.7]
q_high = [ 1.7,  2.,  1.7]
# set the joint limits of the plant
for model in models:
    for joint_index in plant.GetJointIndices(model):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[idx])
            joint.set_position_limits(lower_limits= np.array([q_low[idx]]), upper_limits= np.array([q_high[idx]]))
            idx += 1
        
# construct the RationalForwardKinematics of this plant. This object handles the
# computations for the forward kinematics in the tangent-configuration space
print(plant)
Ratfk = RationalForwardKinematics(plant)
print(plant)
# the point about which we will take the stereographic projections
q_star = np.zeros(3)

#compute limits in t-space
limits_t = []
for q in [q_low, q_high]:
    limits_t.append(Ratfk.ComputeSValue(np.array(q), q_star))

do_viz = True

# This line builds the visualization. Change the viz_role to Role.kIllustration if you
# want to see the plant with its illustrated geometry or to Role.kProximity
visualizer = IrisPlantVisualizer(plant, builder, scene_graph, viz_role=Role.kIllustration)
diagram = visualizer.diagram

# This line will run marching cubes to generate a mesh of the C-space obstacle
# Increase N to increase the resolution of the C-space obstacle.
visualizer.visualize_collision_constraint(N = 60)

#dummy classes to prevent mixups
class SPoint:
    def __init__(self, array:np.ndarray):
        self.ar = array
       

class QPoint:
    def __init__(self, array:np.ndarray):
        self.ar = array
       

INFO:drake:Meshcat listening for connections at http://localhost:7002
INFO:drake:Meshcat listening for connections at http://localhost:7003
/home/peter/git/drake_visiris_build/drake/C_Iris_Examples/iris_plant_visualizer_old.py:71: DrakeDeprecationWarning: (Deprecated) See ForcedPublish() 
*/ (Deprecated.)

Deprecated:
    Use ForcedPublish() instead This will be removed from Drake on or
    after 2023-03-01.
  self.diagram.Publish(self.diagram_context)


In [8]:
sliders = []
sliders.append(widgets.FloatSlider(min=q_low[0], max=q_high[0], value=0, description='q0'))
sliders.append(widgets.FloatSlider(min=q_low[1], max=q_high[1], value=0, description='q1'))
sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = q0.copy()
def handle_slider_change(change, idx):
    q[idx] = change['new']
    visualizer.showres(q)
    #visualizer.visualize_planes()
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)

FloatSlider(value=0.0, description='q0', max=1.7, min=-1.7)

FloatSlider(value=0.0, description='q1', max=2.0, min=-2.0)

FloatSlider(value=0.0, description='q2', max=1.7, min=-1.7)

In [7]:
display(sliders[0])

FloatSlider(value=0.0, description='q0', max=1.7, min=-1.7)

In [48]:
# filter fused joints self collisions so they don't interfere with collision engine
digaram = visualizer.diagram
context = visualizer.diagram_context
sg_context = scene_graph.GetMyContextFromRoot(context)
inspector = scene_graph.model_inspector()

pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()

gids = [gid for gid in inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)]
get_name_of_gid = lambda gid : inspector.GetName(gid)
gids.sort(key=get_name_of_gid)
iiwa_oneDOF_gids = [gid for gid in gids if "iiwa7_oneDOF::" in get_name_of_gid(gid)]
iiwa_twoDOF_gids = [gid for gid in gids if "iiwa7_twoDOF::" in get_name_of_gid(gid)]

oneDOF_fused_col_geom = iiwa_oneDOF_gids[2:]
iiwa_oneDOF_fused_set = GeometrySet(oneDOF_fused_col_geom)
twoDOF_fused_col_geom = iiwa_twoDOF_gids[4:]
iiwa_twoDOF_fused_set = GeometrySet(twoDOF_fused_col_geom)
scene_graph.collision_filter_manager()\
            .Apply(CollisionFilterDeclaration().ExcludeWithin(iiwa_oneDOF_fused_set))
scene_graph.collision_filter_manager()\
            .Apply(CollisionFilterDeclaration().ExcludeWithin(iiwa_twoDOF_fused_set))

# Setup SNOPT IRIS

In [8]:
domain = HPolyhedron.MakeBox( Ratfk.ComputeSValue(q_low, np.zeros((3,))), Ratfk.ComputeSValue(q_high, np.zeros((3,))))
snopt_iris_options = IrisOptions()
snopt_iris_options.require_sample_point_is_contained = True
snopt_iris_options.iteration_limit = 20
snopt_iris_options.configuration_space_margin = 1e-5
#snopt_iris_options.max_faces_per_collision_pair = 60
snopt_iris_options.termination_threshold = -1
#snopt_iris_options.q_star = np.zeros(3)
snopt_iris_options.relative_termination_threshold = 0.05


def SNOPT_IRIS(s_seed : SPoint,  region_obstacles, plant, context, snoptiris_options, Ratforwardkin, default_domain):
    q_seed = Ratforwardkin.ComputeQValue(s_seed.ar.reshape(-1,1), np.zeros((3,1)))
    #print('snopt iris call')
    snoptiris_options.configuration_obstacles = []
    if region_obstacles is not None:
        snopt_iris_options.configuration_obstacles = region_obstacles
    plant.SetPositions(plant.GetMyMutableContextFromRoot(context), q_seed.reshape(-1,1))
    print(plant.GetMyContextFromRoot(context))
    print(plant)
    r = IrisInRationalConfigurationSpace(plant, plant.GetMyContextFromRoot(context), snoptiris_options, domain)
    return r

SNOPT_IRIS_Handle = partial(SNOPT_IRIS,
                            plant = plant,
                            context = visualizer.diagram_context,
                            snoptiris_options = snopt_iris_options,
                            Ratforwardkin = Ratfk,
                            default_domain = domain)

# Run Seeding

In [21]:
# Run Seeding Setup and Guard Phase
seed_s  = SPoint(Ratfk.ComputeSValue(q, np.zeros((3,1))))
# reg = SNOPT_IRIS_Handle(seed_s, region_obstacles=None)
plant.SetPositions(plant.GetMyMutableContextFromRoot(visualizer.diagram_context), np.array(q).reshape(-1,1))
IrisInRationalConfigurationSpace(plant, plant.GetMyMutableContextFromRoot(visualizer.diagram_context), np.zeros((3,1)), snopt_iris_options)

RuntimeError: The seed point is in collision; geometry box_scene::Box1 is in collision with geometry iiwa7_oneDOF::iiwa_oneDOF_link_3_collision

In [19]:
visualizer.diagram_context


In [ ]:
visualizer.diagram_context

In [ ]:
from pydrake.all import Rgba, Sphere, RotationMatrix
def densify_waypoints(waypoints_t):
    densify = 150
    dists = []
    dense_waypoints = []
    for idx in range(len(waypoints_t[:-1])):
        a = waypoints_t[idx]
        b = waypoints_t[idx+1]
        t = np.linspace(1,0, 10)
        locs_endeff = []
        dists_endeff = []
        for tval in t:
            ta = a*tval + b*(1-tval)
            qa = Ratfk.ComputeQValue(ta, np.zeros(7))
            #showres(qa)
            #time.sleep(0.1)            
            visualizer.plant.SetPositions(visualizer.plant_context,qa)
            #visualizer.set_joint_angles(qa)
            tf_tot= plant.EvalBodyPoseInWorld(visualizer.plant_context, visualizer.plant.get_body(pydrake.multibody.tree.BodyIndex(11)))
            tf = tf_tot.translation() + tf_tot.GetAsMatrix4()[:3,:3][:,1] *0.45
            locs_endeff.append(tf)
        for i in range(len(locs_endeff)-1):
            dists_endeff.append(0.1)#)np.linalg.norm(locs_endeff[i]- locs_endeff[i+1]))
        d = np.sum(dists_endeff)
        #print(d * densify)
        t = np.linspace(1,0,int(densify*d))
        for tval in t:
            dense_waypoints.append(waypoints_t[idx]*tval + waypoints_t[idx+1]*(1-tval))
    return dense_waypoints

def plot_endeff_traj(dense_waypoints, opt = True):
    color = Rgba(1,0,0,1.0)
    color2 = Rgba(0,0,1,1.0)
    color3 = Rgba(0,1,1,1.0)
    start_idx = 0
    for i, ta in enumerate(dense_waypoints[::2]):
        qa = Ratfk.ComputeQValue(ta, np.zeros(7))
        #showres(qa)
        #time.sleep(0.1)            
        visualizer.plant.SetPositions(visualizer.plant_context,qa)
        tf_tot= visualizer.plant.EvalBodyPoseInWorld(visualizer.plant_context, plant.get_body(pydrake.multibody.tree.BodyIndex(12)))
        tf = tf_tot.translation() + tf_tot.GetAsMatrix4()[:3,:3][:,1] *0.15

        visualizer.meshcat1.SetObject(f"/iris/points/traj/{i+start_idx}",
                               Sphere(0.005),
                               color)

        visualizer.meshcat1.SetTransform(f"/iris/points/traj/{i+start_idx}",
                                   RigidTransform(RotationMatrix(),
                                                  tf))
        
        tf_tot= visualizer.plant.EvalBodyPoseInWorld(visualizer.plant_context, plant.get_body(pydrake.multibody.tree.BodyIndex(20)))
        tf = tf_tot.translation() + tf_tot.GetAsMatrix4()[:3,:3][:,1] *0.15

        visualizer.meshcat1.SetObject(f"/iris/points/traj2/{i+start_idx}",
                               Sphere(0.005),
                               color2)

        visualizer.meshcat1.SetTransform(f"/iris/points/traj2/{i+start_idx}",
                                   RigidTransform(RotationMatrix(),
                                                  tf))
        
        visualizer.meshcat2.SetObject(f"/iris/points/traj2/{i+start_idx}",
                               Sphere(0.02),
                               color3)

        visualizer.meshcat2.SetTransform(f"/iris/points/traj2/{i+start_idx}",
                                   RigidTransform(RotationMatrix(),
                                                  ta))

In [ ]:
import time

start = poi[np.random.choice(len(poi),1)[0]]
for _ in range(20):
    #nxt = vs.sample_in_regions() #
    while True:
        nxt = poi[np.random.choice(len(poi),1)[0]]
        if nxt[0] != start[0]:
            break
    wp, dist = dspp.solve(start, nxt, refine_path=True)#dijkstra_spp(start, nxt, node_intersections, base_ad_mat, vs.regions, point_conversion, optimize= True)
    print(dist)
    if dist >0:
        dense_waypoints = densify_waypoints(wp)
        plot_endeff_traj(dense_waypoints)
        for ta in dense_waypoints:
            qa = Ratfk.ComputeQValue(ta, np.zeros(3))
            visualizer.showres(qa)
            if visualizer.col_func_handle(qa):
                print(col)
                break
            time.sleep(0.002)
        start = nxt
        time.sleep(0.1)
        visualizer.meshcat1.Delete("/iris/points/traj/")
        visualizer.meshcat1.Delete("/iris/points/traj2/")
        visualizer.meshcat2.Delete("/iris/points/traj2/")
    else:
        nxt = vs.sample_in_regions()